In [1]:
import pandas as pd
import pickle
import numpy as np
import sys
sys.path.insert(0,'../')
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import copy

# Load and prepare data

Deep check

In [2]:
file = open('../resources/memory_mcts_1', 'rb')
memory = pickle.load(file)

memory = memory.ltmemory

print('number of samples:',len(memory))

number of samples: 3680


In [3]:
result = map(lambda x: x['result'], memory) 

result = list(result)

states = map(lambda x: x['state'], memory) 

states = list(states)

y = np.array(result)

X = np.array(states)

positions = list(map(lambda x: list(x['current_position']), memory)) 


positions = np.array(positions)


In [4]:
# y_train = y_train>0
# y_train = y_train.astype(int)
# y_test = y_test>0
# y_test = y_test.astype(int)

y = y>0
y = y.astype(int)

In [5]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [6]:
states = np.array(states)
states.shape

(3680, 48, 8)

In [7]:
positions.shape

(3680, 2)

In [9]:
states = np.array(tf.cast(states, tf.float32))
positions = np.array(tf.cast(positions, tf.float32))

In [10]:
s = np.array( [a.reshape(48,8,1) for a in states])

In [11]:
s.shape

(3680, 48, 8, 1)

In [12]:
X=s

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, shuffle=False)

positions_tr, positions_tst, y_train, y_test = train_test_split(positions, y, test_size=0.1, shuffle=False)

positions_tr, positions_val, y_train, y_val = train_test_split(positions_tr, y_train, test_size=0.1, shuffle=False)

In [14]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model

In [29]:
main_input = Input(shape=(48,8,1),  name='main_input')

x = Conv2D(384, (12, 3), padding='same',activation='relu')(main_input)
x = Conv2D(384, (8, 2), padding='same',activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.0001))(x)
x = MaxPooling2D((4,1))(x)
x = Dropout(0.1)(x)
x = Conv2D(96, (3, 3), padding='same',activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.0001))(x)
x = Dropout(0.1)(x)
x = Flatten()(x)
auxiliary_input = Input(shape=(2,), name='aux_input')
x = keras.layers.concatenate([x, auxiliary_input])
x = Dense(300, activation='elu', kernel_regularizer=keras.regularizers.l2(l=0.0001))(x)
x = Dropout(0.2)(x)
x = Dense(400, activation='elu', kernel_regularizer=keras.regularizers.l2(l=0.0001))(x)
x = Dropout(0.2)(x)
x = Dense(200, activation='elu', kernel_regularizer=keras.regularizers.l2(l=0.0001))(x)
main_output = Dense(1, activation='sigmoid', name='main_output')(x)
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output])

model.compile(optimizer='adam', loss='binary_crossentropy')

In [30]:
for e in range(1):
    model.fit([X_train, positions_tr], [y_train], epochs=1, batch_size=32)
#     y_pred = model.predict_classes([X_val, positions_val])

    y_pred = model.predict([X_val, positions_val])

    mae = mean_absolute_error(y_val, y_pred)   
    
    print(mae)

Train on 2980 samples
2980/2980 [==============================] - 60s 20ms/sample - loss: 0.7895
0.4001900503434331


In [31]:
    y_pred = model.predict([X_test, positions_tst])

    mae = mean_absolute_error(y_test, y_pred)   
    
    print(mae)

0.42735722353277


In [32]:
model.save('../resources/Residual_CNN_mcts.h5') 

In [33]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 48, 8, 1)]   0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 48, 8, 384)   14208       main_input[0][0]                 
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 48, 8, 384)   2359680     conv2d_15[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 12, 8, 384)   0           conv2d_16[0][0]                  
____________________________________________________________________________________________

In [ ]:
# params={
#         'epochs':20, 'batch_size':[32], 'shuffle':[True], 
#         'activation_conv':['elu',], 'activation_dense':['sigmoid'], 'optimizer':['adam','sgd'],
#     'architectures':[
#         1
#     ]} 
#     best_model_acc = None
#     best_acc = 0
#     best_par_acc = []
#     best_model_mae = None
#     best_mae = 1
#     best_par_mae = []
#     dict_grid = {}
#     X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)
#     for bs in params['batch_size']:
#         for s in params['shuffle']:
#             for o in params['optimizer']:
#                 for ah in params['activation_hidden']:
#                     for a in params['activation']:
#                         for arch in params['architectures']:
#                             main_input = Input(shape=(48,8,1),  name='main_input')

#                             x = Conv2D(384, (8, 2), padding='same',activation='elu')(main_input)
#                             x = MaxPooling2D(pool_size=(2, 2))
#                             x = Conv2D(96, (8, 2), padding='same',activation='elu')(main_input)
#                             x = Dropout(0.25)(x)
#                             x = Conv2D(96, (4, 2), padding='same',activation='elu')(main_input)
#                             x = Dropout(0.25)(x)
#                             x = Flatten()(x)
#                             auxiliary_input = Input(shape=(2,), name='aux_input')
#                             x = Dense(200, activation='elu')(x)
#                             x = Dense(100, activation='elu')(x)
#                             main_output = Dense(1, activation='sigmoid', name='main_output')(x)
#                             model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output])

#                             model.compile(optimizer='adam', loss='binary_crossentropy', metric=['mae'], validation_split=0.2)

In [ ]:
model = tf.keras.models.load_model('../resources/Residual_CNN_mcts.h5')

In [ ]:
model.summary()